In [1]:
#csv desde drive
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
import numpy as np

Using TensorFlow backend.


## 3. *Encoder-Decoder* sobre Texto

Trabajos recientes en redes neuronales han demostrado que se puede aplicar a problemas bastante complejos gracias a la flexibilidad la definición de las redes, además de que se pueden adaptar a distintos tipos de datos brutos (dominios). Con el objetivo de explorar el enfoque anterior de *traducción* de algun tipo de dato, en esta sección deberá realizarlo con texto para traducción de un lenguaje humano a otro (e.g. inglés a alemán, chino a ruso).

In [3]:
df = pd.read_csv("./por.txt", sep="\t", names=["Source","Target"])

> a) Visualice los datos ¿Qué es la entrada y qué es la salida? Comente sobre los múltiples significados/sinónimos que puede tener una palabra al ser traducida y cómo propondría arreglar eso. *se espera que pueda implementarlo*

In [4]:
df.head(10)

,Source,Target
0,Go.,Vai.
1,Go.,Vá.
2,Hi.,Oi.
3,Run!,Corre!
4,Run!,Corra!
5,Run!,Corram!
6,Run.,Corre!
7,Run.,Corra!
8,Run.,Corram!
9,Who?,Quem?


La entrada del dataset corresponde a la frase o palabra escrita en ingles y el objetivo o salida corresponde a la traducción de lo anterior al idioma que se desea trabajar, en este caso portugues.

Al inicio del dataset se pueden ver unicamente palabras junto con su traducción explicita, se puede notar ademas que existen palabras repetidas en ingles que se traducen de una manera distinta, esto se debe a que el ingles no posee una conjugación explicita en la palabra como es el caso del español o el portuges, sino que se le da un contexto con el resto de la oración. Se espera que el encoder tenga problemas al traducir frases con dichas palabras.

In [5]:
df.tail()

,Source,Target
135666,"The Tatoeba Project, which can be found online...","O Projeto Tatoeba, que se pode encontrar on-li..."
135667,No matter how much you try to convince people ...,Não importa o quanto você tenta convencer os o...
135668,Some movies make such an impact that one never...,Alguns filmes são tão marcantes que jamais nos...
135669,A child who is a native speaker usually knows ...,Uma criança que é falante nativa geralmente sa...
135670,We recommend adding sentences and translations...,Recomendamos acrescentar frases e traduções na...


Cuando se analizan los datos al final del conjunto se puede ver que ya se esta trabajando con oraciones y no unicamente con palabras simples, se espera que estos ejemplos permitan a la red neuronal aprender a reconocer un poco el contexto en el cual se esta utilizando la palabra.

> b) Realice un pre-procesamiento a los textos como se acostumbra para eliminar símbolos inecesarios u otras cosas que estime conveniente, comente sobre la importancia de éste paso. Además de ésto deberá agregar un símbolo al final de la sentencia *target* para indicar un "alto" cuando la red neuronal necesite aprender a generar una sentencia.

In [6]:
table = str.maketrans('', '', string.punctuation)

def clean_text(text, where=None):
    """ OJO: Sin eliminar el significado de las palabras."""
    text = text.lower()
    tokenize_text = text.split()
    tokenize_text = [word.translate(table) for word in tokenize_text]#eliminar puntuacion
    tokenize_text = [word for word in tokenize_text if word.isalpha()] #remove numbers
    if where =="target":
        tokenize_text = tokenize_text + ["#anvorgesa"] 
    return tokenize_text

  
texts_input = list(df['Source'].apply(clean_text))
texts_output = list(df['Target'].apply(clean_text, where='target'))

> Cree un conjunto de validación y de pruebas fijos de $N_{exp} = 10000$ datos ¿Cuántos datos quedan para entrenar? 

In [7]:
N_exp = 10000

In [8]:
X_train_l, X_test_l, Y_train_l, Y_test_l = train_test_split(texts_input, texts_output,
                                                            test_size=N_exp, random_state=22)
X_train_l, X_val_l, Y_train_l, Y_val_l = train_test_split(X_train_l, Y_train_l, 
                                                          test_size=N_exp, random_state=22)

In [9]:
len(X_train_l)

115671

Luego de haber generado el cconjunto de testing y validación aún quedan 115671 ejemplos para entrenar el modelo.

> *Recuerde que si no puede procesar los datos de entrenamiento adecuadamente siempre puede muestrear en base a la capacidad de cómputo que posea*

> c) Genere un vocabulario, **desde el conjunto de entrenamiento**, sobre las palabras a recibir y generar en la traducción, esto es codificarlas a un valor entero que servirá para que la red las vea en una representación útil a procesar, *comience desde el 1 debido a que el cero será utilizado más adelante*. Para reducir el vocabulario considere las palabras que aparecen un mínimo de *min_count* veces en todo los datos, se aconseja un valor de 3. Comente sobre la importancia de ésto al reducir el vocabulario ¿De qué tamaño es el vocabulario de entrada y salida? ¿La diferencia de ésto podría ser un factor importante?

In [10]:
def create_vocab(texts, min_count=1):
    count_vocab = {}
    for sentence in texts:
        for word in sentence:
            if word not in count_vocab:
                count_vocab[word] = 1
            else:
                count_vocab[word] += 1
    return [word for word,count in count_vocab.items() if count >= min_count]
  
vocab_source = create_vocab(X_train_l, min_count=3)
word2idx_s = {w: i+1 for i, w in enumerate(vocab_source)} #index (i+1) start from 1,2,3,...
idx2word_s = {i+1: w for i, w in enumerate(vocab_source)}
n_words_s = len(vocab_source)
vocab_target = create_vocab(Y_train_l, min_count=3)
word2idx_t = {w: i+1 for i, w in enumerate(vocab_target)}  #Converting text to numbers
idx2word_t = {i+1: w for i, w in enumerate(vocab_target)}
n_words_t = len(vocab_target)

Dado que existe una gran cantidad de palabras dentro del vocabulario y la memoria del sistema que se utilice no es ilimitada es necesario reducir el tamaño para poder trabajar con el dataset, ademas palabras que aparescan menos de una determinada cantidad de veces pueden considerarse como poco relevantes para la traducción del texto.

In [11]:
print("Tamaño de palabras en origen:",n_words_s)
print("\nTamaño de palabras en destino",n_words_t)

Tamaño de palabras en origen: 5954

Tamaño de palabras en destino 8760


El tamaño del conjunto de palabras de origen es de 5954 palabras, mientras que el destino es de 8760, esto se debe a lo comentado anteriormente referenciando a que el ingles obtiene la conjugación de una palabra en base al contexto en el cual se esta hablando y no explicitamente en la forma en que se escribe.

Esto, en opinión propia, puede afectar negativamente a la red, puesto que sin el contexto adecuado una red podría traducir una palabra de multiples maneras.

> Ahora codifique las palabras a los números indexados con el vocabulario. Recuerde que si una palabra en los otros conjuntos, o en el mismo de entrenamiento, no aparece en el vocabulario no se podrá generar una codificación, por lo que será **ignorada** ¿Cómo se podría evitar ésto?

In [12]:
""" Source/input data """
dataX_train = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_train_l]
dataX_val = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_val_l]
dataX_test = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_test_l]

""" Target/output data """
dataY_train = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_train_l]
dataY_val = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_val_l] 
dataY_test = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_test_l] 

> d) Debido al largo variable de los textos de entrada y salida será necesario estandarizar ésto para poder trabajar de manera más cómoda en Keras, *cada texto (entrada y salida) pueden tener distinto largo máximo*. Comente sobre la decisión del tipo de *padding*, *pre o post* ¿Qué sucede al variar el largo máximo de instantes de tiempo para procesar en cada parte del modelo (entrada y salida)?

In [13]:
""" INPUT DATA (Origin language) """
max_inp_length = max([max(map(len,dataX_train)), max(map(len,dataX_val)), max(map(len,dataX_test))])
print("Largo max inp: ",max_inp_length)
word2idx_s["#anvorgesa"] = 0 #padding symbol
idx2word_s[0] = "#anvorgesa"
n_words_s += 1  
X_train = sequence.pad_sequences(dataX_train, maxlen=max_inp_length, padding='post', value=word2idx_s["#anvorgesa"])
X_val = sequence.pad_sequences(dataX_val, maxlen=max_inp_length, padding='post', value=word2idx_s["#anvorgesa"])
X_test = sequence.pad_sequences(dataX_test, maxlen=max_inp_length, padding='post', value=word2idx_s["#anvorgesa"])

""" OUTPUT DATA (Destination language) """
max_out_length = max([max(map(len,dataY_train)), max(map(len,dataY_val)), max(map(len,dataY_test))])
print("Largo max out: ",max_out_length)
word2idx_t["#anvorgesa"] = 0 #padding symbol
idx2word_t[0] = "#anvorgesa"
n_words_t += 1  
Y_train = sequence.pad_sequences(dataY_train, maxlen=max_out_length, padding='post', value=word2idx_t["#anvorgesa"])
Y_val = sequence.pad_sequences(dataY_val, maxlen=max_out_length, padding='post', value=word2idx_t["#anvorgesa"])
Y_test = sequence.pad_sequences(dataY_test, maxlen=max_out_length, padding='post', value=word2idx_t["#anvorgesa"])


Largo max inp:  35
Largo max out:  34


In [14]:
Y_train[0]

array([1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

> e) Para evitar que la red obtenga una ganancia por imitar/predecir el símbolo de *padding* que está bastante presente en los datos coloque un peso sobre éste clase, con valor 0, así se evita que tenga impacto en la función objetivo. Ya que *keras* no soporta directamente ésto en series de tiempo coloque el peso a cada instante de tiempo de cada dato de entrenamiento dependiendo de su clase. Comente sobre alguna otra forma en que se podría manejar el evitar que la red prediga en mayoría el símbolo de *padding*.

In [15]:
c_weights = np.ones(n_words_t)
c_weights[0] = 0 #padding class masked
sample_weight = np.zeros(Y_train.shape)
for i in range(sample_weight.shape[0]):
    sample_weight[i] = c_weights[Y_train[i,:]]

> f) Para lograr la tarea defina una red recurrente del tipo *encoder*-*decoder* como la que se presenta en la siguiente imágen.
<img src="https://chunml.github.io/ChunML.github.io/images/projects/sequence-to-sequence/repeated_vector.png" width="60%" />
En primer lugar defina el *Encoder* que procesara el texto de entrada y retornará un solo vector final, haciendo uso de las capas ya conocidas de *Embedding* para generar un vector denso de palabra y *GRU*, pero en su versión acelerada para GPU.

In [16]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU, GRU
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(input_dim=n_words_s, output_dim=EMBEDDING_DIM, input_length=max_inp_length))
#model.add(CuDNNGRU(64, return_sequences=True))
#model.add(CuDNNGRU(128, return_sequences=False))
model.add(GRU(64, return_sequences=True))
model.add(GRU(128, return_sequences=False))

Instructions for updating:
Colocations handled automatically by placer.


> Luego defina la sección que conecta el largo (*timesteps*) de entrada *vs* el de salida.

In [17]:
from keras.layers import RepeatVector
model.add(RepeatVector(max_out_length)) #conection

Finalmente defina el *Decoder* para generar la secuencia de salida en texto de palabras en otro idioma, a través de la función *softmax* sobre cada instante de tiempo (*timestep*). 

In [18]:
from keras.layers import GRU, TimeDistributed,Dense
#model.add(CuDNNGRU(128, return_sequences=True))
#model.add(CuDNNGRU(64, return_sequences=True))
model.add(GRU(64, return_sequences=True))
model.add(GRU(128, return_sequences=True))
model.add(TimeDistributed(Dense(n_words_t, activation='softmax')))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 35, 100)           595500    
_________________________________________________________________
gru_1 (GRU)                  (None, 35, 64)            31680     
_________________________________________________________________
gru_2 (GRU)                  (None, 128)               74112     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 34, 128)           0         
_________________________________________________________________
gru_3 (GRU)                  (None, 34, 64)            37056     
_________________________________________________________________
gru_4 (GRU)                  (None, 34, 128)           74112     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 34, 8761)          1130169   
Total para

> Entrene la red entre 1 a 5 *epochs*, agregando los pesos definidos sobre cada ejemplo de entrenamiento. Además de utilizar una función de pérdida que evita generar explícitamente los *one hot vector*


Antes de entrenar es necesario cambiar la forma del input y output a 3 dimensiones

In [19]:
Y_train = np.reshape(Y_train, (Y_train.shape[0], max_out_length, 1))
Y_val = np.reshape(Y_val, (Y_val.shape[0], max_out_length, 1))
Y_test = np.reshape(Y_test, (Y_test.shape[0], max_out_length, 1))

Y_train.shape
#X_test = np.reshape(X_test, (X_test.shape[0], lstm_num_timesteps, lstm_num_features))

(115671, 34, 1)

In [20]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', sample_weight_mode='temporal')
model.fit(X_train, Y_train, epochs=1, batch_size=256,validation_data=(X_val, Y_val), sample_weight = sample_weight)

Instructions for updating:
Use tf.cast instead.
Train on 115671 samples, validate on 10000 samples
Epoch 1/1
115671/115671 [==============================] - 1911s 17ms/step - loss: 5.8284 - val_loss: 13.5294


In [21]:
model.save('./modell.h5')